In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/docterm/dtm.pkl
/kaggle/input/cvstop/cv_stop.pkl
/kaggle/input/doctermm/dtm2.pkl
/kaggle/input/preprocc-data/preprocessed_data.pkl


In [2]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('/kaggle/input/doctermm/dtm2.pkl')
data

,ago,anxiety,bad,better,control,day,days,did,didnt,doctor,...,taking,time,took,week,weeks,weight,went,work,year,years
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213864,0,2,0,0,0,4,0,0,0,0,...,1,1,1,1,0,0,1,1,0,0
213865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
213866,1,1,1,2,0,1,0,0,1,0,...,0,1,1,0,0,0,0,1,0,1
213867,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
# tdm.head()

NameError: name 'tdm' is not defined

In [5]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,0,1,2,3,4,5,6,7,8,9,...,213859,213860,213861,213862,213863,213864,213865,213866,213867,213868
ago,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
anxiety,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,1,0,0
bad,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
better,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,2,0,0
control,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# The Gensim corpus is a collection of sparse vectors, 
# where each vector represents a document and contains tuples of (word_index, word_frequency) pairs. 

In [6]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm) # stores large sparse matrix
corpus = matutils.Sparse2Corpus(sparse_counts)

In [7]:
 # Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("/kaggle/input/cvstop/cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [8]:
cv

CountVectorizer(stop_words='english')

**Latent Dirichlet Allocation (LDA) topic modeling.** 

corpus: dtm
id2word: maps word id to word
passes: iterations

In [ ]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=10)
lda.print_topics()

# Topic Modelling(Nouns)

1. tokenize
2. pos_tag: part of speech that the text is noun,verb,adjective etc
3. is_nouns(pos): filters nouns

In [ ]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [ ]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('/kaggle/input/preprocc-data/preprocessed_data.pkl')
data_clean

In [ ]:
data_clean.columns

In [ ]:
data_nouns = pd.DataFrame(data_clean['clean_review'].apply(nouns))

In [ ]:
# import pandas as pd
# from sklearn.feature_extraction import text
# from sklearn.feature_extraction.text import CountVectorizer

# # Assuming you have already defined the nouns function and loaded the data

# # Apply the nouns function to the transcripts to filter only on nouns
# data_nouns = pd.DataFrame(data_clean.review_clean.apply(nouns))

# # Re-add the additional stop words since we are recreating the document-term matrix
# add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
#                   'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
# stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# # Convert frozenset to list
# stop_words = list(stop_words)

# # Recreate a document-term matrix with only nouns
# cvn = CountVectorizer(stop_words=stop_words)
# data_cvn = cvn.fit_transform(data_nouns['review_clean'])
# data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
# data_dtmn.index = data_nouns.index
# data_dtmn

<h2>creating dtm with only nouns</h2>

In [ ]:
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Assuming you have already defined the nouns function and loaded the data

# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.review_clean.apply(nouns))

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']

stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Convert frozenset to list
stop_words = list(stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns['review_clean'])
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

In [ ]:
# Transpose the DataFrame
data_dtmn_transposed = data_dtmn.transpose()

# Display the transposed DataFrame
print(data_dtmn_transposed)


In [ ]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [ ]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

In [ ]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

extracting both nouns and ajectives

In [ ]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [ ]:
data_nouns_adj = pd.DataFrame(data_clean['clean_review'].apply(nouns_adj))
data_nouns_adj

In [ ]:
print(data_nouns_adj.columns)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
# # Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
# cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
# data_cvna = cvna.fit_transform(data_nouns_adj['clean_review'])
# data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
# data_dtmna.index = data_nouns_adj.index
# data_dtmna

# nouns+adjective, removing most common words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Define additional stop words
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']

# Concatenate additional stop words with the default English stop words
stop_words = list(text.ENGLISH_STOP_WORDS) + add_stop_words

# Initialize CountVectorizer with stop words and max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)

# Fit and transform the data using only nouns and adjectives
data_cvna = cvna.fit_transform(data_nouns_adj['clean_review'])

# Create a DataFrame from the document-term matrix
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())

# Set the index of the DataFrame
data_dtmna.index = data_nouns_adj.index

# Display the DataFrame
data_dtmna

In [ ]:
# # Transpose the DataFrame
# data_dtmna_transposed = data_dtmna.transpose()

# # Display the transposed DataFrame
# print(data_dtmna_transposed)


In [ ]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

In [ ]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=40)
ldana.print_topics()

# FINAL

In [ ]:
for idx, topic in ldana.print_topics(-1):
    print("Topic {}: {}".format(idx + 1, topic))

In [ ]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]

# Unpack the values from corpus_transformed
topics_per_transcript = [[a for (a, b) in doc] for doc in corpus_transformed]

# Combine with transcript indices
topics_with_indices = list(zip(topics_per_transcript, data_dtmna.index))

# Print the result
for idx, topic in ldana.print_topics(-1):
     print("Topic {}: {}".format(idx + 1, topic))

In [ ]:
# #part 2
# # Function to tokenize and filter for verbs
# def verb(text):
#     '''Given a string of text, tokenize the text and pull out only the verbs.'''
#     is_verb = lambda pos: pos[:2] == 'VB'
#     tokenized = word_tokenize(text)
#     all_verb = [word for (word, pos) in pos_tag(tokenized) if is_verb(pos)]
#     return ' '.join(all_verb)

# # Apply the nouns function to the transcripts to filter only on nouns
# data_verb = pd.DataFrame(data_clean['clean_review'].apply(verb))
# data_verb

# # Create a new document-term matrix using only nouns, adjectives, and verbs, also remove common words with max_df
# cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
# data_cvna = cvna.fit_transform(data_verb.transcript)
# data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
# data_dtmna.index = data_verb.index

# # Create the gensim corpus
# corpus_verb = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# # Create the vocabulary dictionary
# id2word_verb = dict((v, k) for k, v in cvna.vocabulary_.items())

# # Attempt LDA with verbs
# lda_verb = models.LdaModel(corpus=corpus_verb, num_topics=4, id2word=id2word_verb, passes=10)
# for idx, topic in ldana.print_topics(-1):
#     print("Topic {}: {}".format(idx + 1, topic))